# Prueba del modelo largo de lenguaje Dolly-v2-3b

1. Modelo de lenguaje entrenado: Dolly-v2-3b
1. Numero de parametro que posee: 3 Billones
1. Tipo de entrenamiento: Modelo Instruction-tuned
1. Link: https://huggingface.co/databricks/dolly-v2-3b

Para poder utilizar el modelo entrenado de Dolly se necesitan instalar liberías en el sistema operativo... pero, este sistema es una máquina virtual que está limitada por el tiempo permitido de uso de un procesador

Se necesita un procesador que permita realizar operaciones especiales con el modelo, es necesario un acelerador de procesamiento gráfico, como el de las tarejetas de aceleración gráfica para los videojuegos, se debe elegir T4 GPU en el entorno de ejecución.

# Herramientas de apoyo

**Transformers** es una librería que facilita el uso de los modelos largos de lenguajes, **sentencepiece** es otra herramientaque que permite codificar y decodificar en tokens las consultas que se le hacen a un modelo largo de lenguaje, y **accelerate** permite detectar automáticamente el tipo de configuración distribuida que se tiene disponible en una máquina virtual e inicializará todos los componentes necesarios para el uso de un modelo largo de lenguaje.

Estas herramientas actualmente ya están instaladas en el entorno virtual, y por eso ya no se debe proceder a instalar las liberías para su uso con Python, eso se antes se hacía con el administrador de paquetes pip

```
!pip install transformers
!pip install sentencepiece
!pip install accelerate
```

# Integrar las herramientas necesarias

Teniendo las herramientas listas para usarse en los códigos fuente de los programas de Python ya se puede utilizar las liberías, para ello se importan en los programas a crear para su uso

In [ ]:
# Importar librerías
import torch                        # Activar computación de Tensores con aceleración de GPU
from transformers import pipeline   # Atajo para usar modelos pre-entrenados para inferencia
import os                           # Permite interactuar con instrucciones de entrada y salida del sistema operativo

# Descarga del modelo de Dolly

Con las herramientas para trabajar modelos largos de lenguaje falta tener un modelo para usar, así que se debe descargar un modelo y usar una de las herramientas para conectarse e interactuar con él

In [ ]:
# Descarga el modelo de dolly y carga el modelo en la variable dolly_pipeline
dolly_pipeline = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")

# Facilitando el uso del modelo Dolly

Para facilitar la interacción con el modelo sin utilizar parámetros complejos y preparaciones cada vez que se quiera utilizar es buena idea crear una función, una forma de consultar al modelo las preguntas que se quieran y obtener una respuesta que se pueda entender

In [ ]:
# Definir una función con la cual preguntarle al modelo mediante una consulta
def preguntar_a_dolly( consulta ):
    # Se usa una variable para definir el token fundador
    token_fundador = """
    Eres un experto en matemática.
    Eres muy bueno explicando conceptos de matemática.
    Ayuda al usuario dando una respuesta comprensible.
    """
    # Se crea una variable auxiliar para mostrar lineas divisorias
    divisor = f"-------------------------------------------\n"
    # Se utiliza la tubería para preguntarle a dolly uniendo el token fundador con la consulta
    dolly_responde = dolly_pipeline( token_fundador + consulta, max_new_tokens = 500 )
    # La respuesta queda almacenada en un arreglo, por lo que se guarda solo el texto generado
    respuesta_generada = dolly_responde[ 0 ][ "generated_text" ]
    # Se muestran los textos de manera ordenada y creativa para depurado del uso
    print( f"{divisor}Token fundador     : {token_fundador}"       )
    print( f"{divisor}Consulta           : \n{consulta}"           )
    print( f"{divisor}Respuesta de Dolly : \n{respuesta_generada}" )
    # se devuelve solo la respuesta para usos adicionales hacia donde fue solicitada
    return respuesta_generada

# Probando el funcionamiento

Cuando se usa una interfaz de IA del tipo de modelo largo de lenguaje se usa un texto al que denominamos prompt (*una instrucción de entrada*) que se le envía al modelo, y del cual se obtiene una respuesta, para probar ese comportamiento se codificará con variables la interacción con el modelo a travez de la función creada

In [ ]:
# Crear el prompt de consulta
prompt = "¿Qué es un tensor en matemática?"
# Obtener la respuesta de dolly
respuesta = preguntar_a_dolly( prompt )

La prueba es buena, se pueden hacer mas pruebas cambiando en la función el token generador (*modificandolo y ejecutando el bloque*) y luego modificando el prompt en la prueba (*y ejecutando el bloque*)

# Lanzándola a la web

Las formas en las que actualmente usamos las IA es mediante interfaces gráficas web, para que sea más práctico y que más personas puedan usarla

# Instalando la herramienta para crear aplicaciones web

Se necesitará una herramienta que facilite crear interfaces amigables de tipo web, y **gradio** es otra librería que se puede utilizar para facilitar esa tarea, se instala con el administrador de paquetes pip una versión en particular

In [ ]:
!pip install gradio

# Integrando las herramientas para la web

Para utilizarla se importará como una libería, pero también se necesita otra librería especial para indicarle que la codificación del texto debe ser aceptable y con símbolos reconocibles en la web (*UTF-8*)

In [ ]:
# Importar
import locale        # Librería de configuración formato de carácteres para la web
import gradio as gr  # Librería que facilita la creación de aplicaciones web

# Eligiendo la codificación de textos de la aplicación web

Se debe indicar que se usará la codificación especial UTF-8 a todos los textos que se generen para ser mostrados y utilizados en las interfaces web.

In [ ]:
# Definir la codificación de caractéres a utilizar con el formato UTF-8
locale.getpreferredencoding = lambda: "UTF-8"


# Diseñando la interfaz de la aplicación web

Para poder crear una interfaz web lo correcto es diseñar primero la idea:
* Se necesitará inicialmente un espacio (textarea) para poder escribir el prompt de entrada
* También deberá existir un espacio (textarea) para que se pueda leer la respuesta
* Asingar un título no es mala idea, identificará a la aplicación web
* Y también una descripción de la aplicación para darle elegancia
* Se puede indicar como serían unos ejemplos de prompt a utiizar

Todos estos elementos serán útiles, pero realmente, lo mas importante es saber a que función se le hará responsable de recibir los valores, y que también nos devolverá una respuesta, la función a utilizar será la que se denominó como **preguntar_a_dolly**



In [ ]:
# Diseño de la aplicación web
# Una variable que definirá una caja para introducir texto
caja_de_entrada  = gr.Textbox( label = "Escribe Prompt Aqui" , lines = 6 )
# Otra variable que definirá una caja para mostrar texto
caja_de_salida   = gr.Textbox( label = "Respuesta"           , lines = 3 )
# Una variable más para darle un título a la aplicación web
titulo_de_mi_app = "Mi tutor IA de matemáticas"
# Podría ser útil usar una variable para mostrar la descripción de la aplicación
descripcion_app  = "Herramienta para aprender de matemáticas con Dolly"
# Un conjunto de ejemplos para poder utilizar como entradas de textos
ejemplos         = [
    "Explicame el teorema de Pitágoras",
    "Explícame la suma de Gauss"
]
# La agrupación de todo en una sola interfaz de gradio
interfaz = gr.Interface(
    fn            = preguntar_a_dolly   ,
    inputs        = [ caja_de_entrada ] ,
    outputs       = [ caja_de_salida  ] ,
    title         = titulo_de_mi_app    ,
    description   = descripcion_app     ,
    examples      = ejemplos            ,
    flagging_mode = "never"               # Desaparece un botón llamado Flag
)

# Utilizando la aplicación web



El diseño que ya está listo debe ser ejecutado y puede ser también publicado, la herramienta grado permite utilziar la interfaz creada a travez de un tunel, publicando la interfaz y haciendola accesible a mas usuarios a través de Internet por lo que brindará una url para poder compartirla

In [ ]:
# Lanzamiento de la aplicación web
interfaz.launch( share = True )

# Finalizando la aplicación web

Cuando ya se ha probado la aplicación es correcto detener el servicio para que no se desperdicie el tiempo de uso, tanto de las unidades de CPU de Google Colab, como el servicio de publicación de la herramienta del tunel al sitio de gradio

In [ ]:
# Detener la aplciación web
interfaz.close()

# Probando otra interfaz

Gradio tiene la facultad de proveer otro tipo de interfaces, existe una interfaz tipo chat, la cual sería mas familiar para cuando se trabaja con un modelo largo de lenguaje, pero necesita una funcion que recibe dos parámetros, los parámetros son el mensaje y el historial de mensajes

## Creando una función envolvedora

Crearemos una función que nos dará las repsuestas, pero recibirá dos parámetros y no utilizará el historial en este ejercicio simple

In [ ]:
def respuestas(mensaje, historial):
    return preguntar_a_dolly(mensaje)

## Diseñando la interfaz de chat

Luego crearemos una interfaz de gradio... ocupará muy pocos parámetros

In [ ]:
chat = gr.ChatInterface(
    respuestas      ,
    type="messages" ,
    autofocus=False
)

##Lanzando el chat

Y lanzaremos la interfaz para probarlo

In [ ]:
chat.launch( share = True)

## Finalizando el chat

No hay que olvidar cerrar la interfaz cuando ya no la usemos

In [ ]:
chat.close()